## Hyper parameter tuning by SageMaker

1. OS level setting
2. Prepare Requirements
3. Hyperparameter Tuning

**Reference**

* [Hyperparameter Tuning using SageMaker PyTorch Container](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/pytorch_mnist/hpo_pytorch_mnist.ipynb)

## OS level setting

Install packages.

```
sudo yum install gcc72-c++.x86_64
sudo yum install clang
```

g++: Install & link same version of gcc.

## Prepare Requirements

In [27]:
!git pull origin master
!pip install pipenv

From https://github.com/icoxfog417/allennlp-sagemaker-tuning
 * branch            master     -> FETCH_HEAD
Already up-to-date.
en-core-web-sm 2.1.0 requires spacy>=2.1.0, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [28]:
! export PIPENV_VENV_IN_PROJECT=1 && cd ../ && pipenv install --python=3.6

Virtualenv already exists!
Removing existing virtualenv…
Creating a virtualenv for this project…
Pipfile: /home/ec2-user/SageMaker/allennlp-sagemaker-tuning/Pipfile
Using /home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/python (3.6.8) to create virtualenv…
⠧ Creating virtual environment...Using base prefix '/home/ec2-user/anaconda3/envs/JupyterSystemEnv'
New python executable in /home/ec2-user/SageMaker/allennlp-sagemaker-tuning/.venv/bin/python
Installing setuptools, pip, wheel...
done.
Running virtualenv with interpreter /home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/python

✔ Successfully created virtual environment! 
Virtualenv location: /home/ec2-user/SageMaker/allennlp-sagemaker-tuning/.venv
  $ pipenv --rm and rebuilding the virtual environment may resolve the issue.
  $ pipenv check will surely fail.
Installing dependencies from Pipfile.lock (44ebd7)…
  🐍   ▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉ 108/108 — 00:01:33mmmmmmmmmmmmmmmmmmmmmmmmmmmmmm
To activate this project's virtual

In [29]:
import os


def set_pythonpath():
    import sys
    python_version = "python" + str(sys.version_info.major) \
                     + "." + str(sys.version_info.minor)
    venv_dir = "../.venv/lib/{}/site-packages".format(python_version)
    lib_dir = os.path.join(os.path.realpath("."), venv_dir)
    project_dir = os.path.join(os.path.realpath("."), "../")
    sys.path.append(lib_dir)
    sys.path.append(project_dir)

set_pythonpath()

In [30]:
def execute_example():
    from example.train import train as train_fn
    
    root = 'https://raw.githubusercontent.com/allenai/allennlp/master/tutorials/tagger/'
    train_data_path = root + 'training.txt'
    validation_data_path = root + 'validation.txt'

    embedding_dim = 6
    hidden_dim = 6
    num_epochs = 1

    train_fn(train_data_path, validation_data_path,
                   embedding_dim, hidden_dim, num_epochs=num_epochs)

In [31]:
execute_example()

2it [00:00, 3045.97it/s]
2it [00:00, 6558.72it/s]
100%|██████████| 4/4 [00:00<00:00, 21873.81it/s]
ERROR:allennlp.common.util:unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/allennlp-sagemaker-tuning/notebooks/../.venv/lib/python3.6/site-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    encoding='utf-8')
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.
accuracy: 0.2222, loss: 1.2112 ||: 100%|██████████| 1/1 [00:00<00:00, 151.60it/s]
accuracy: 0.2222, loss: 1.1984 ||: 100%|██████████| 1/1 [00:00<00:00, 625.46it/s]


['V', 'V', 'V', 'V', 'V']


## Hyperparameter Tuning

### Create Session

In [32]:
import sagemaker
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner


sagemaker_session = sagemaker.Session()
bucket = "sagemaker.tech-sketch.jp"
prefix = "allennlp_test"
role = sagemaker.get_execution_role()
role = role.replace("service-role/", "")

### Upload data

In [33]:
from allennlp.common.file_utils import cached_path


root = "https://raw.githubusercontent.com/allenai/allennlp/master/tutorials/tagger/"
urls = [(root + file_name) for file_name in ("training.txt", "validation.txt")]
paths = [cached_path(u) for u in urls]
s3_paths = []

for path in paths:
    s3_path = sagemaker_session.upload_data(path=path, bucket=bucket, key_prefix=prefix)
    print("input spec (in this case, just an S3 path): {}".format(s3_path))
    s3_paths.append(s3_path)

input spec (in this case, just an S3 path): s3://sagemaker.tech-sketch.jp/allennlp_test/c3e1f451545a79cf7582dec24d072db6f5bb0d1ae24a924d03c9944516e16b60.47b1193282cbd926a1b602cc6d5a22324cfab24e669ca04f1ff4851a35c73393
input spec (in this case, just an S3 path): s3://sagemaker.tech-sketch.jp/allennlp_test/a377491818b2bbd2f0561346da1d8d25f29bbc1c8df640eaf6ee125071d18d16.22d6cc9ff0fe67add48c843670f9b158a2cd4d4527d8d3b9587a7c48ff356e2f


### Make Estimator

In [34]:
from sagemaker.pytorch import PyTorch


def from_root(path):
    root = os.path.join(os.path.realpath("."), "../")
    return os.path.abspath(os.path.join(root, path))

estimator = PyTorch(entry_point="tuning.py",
                                    source_dir="../../allennlp-sagemaker-tuning",
                                    dependencies=[from_root("example"), from_root(".venv")],
                                    role=role,
                                    framework_version="1.0.0",
                                    train_instance_count=1,
                                    train_instance_type="ml.p2.8xlarge",
                                    hyperparameters={
                                        "train-file-name": os.path.basename(s3_paths[0]),
                                        "validation": os.path.basename(s3_paths[1]),
                                        "epochs": 10
                                    })

### Define Parameter Range

In [35]:
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.01, 0.1),
    "embedding-dim": CategoricalParameter([6, 12]),
    "hidden-dim": CategoricalParameter([6, 12])
}

### Define Target Metrics

In [36]:
objective_metric_name = "validation loss"
objective_type = "Minimize"
metric_definitions = [
    {"Name": objective_metric_name,
      "Regex": "validation_loss=([0-9\\.]+)"}
]

### Create Tuner

In [37]:
tuner = HyperparameterTuner(
                            estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=3,
                            max_parallel_jobs=1,
                            objective_type=objective_type)

### Execute Tuning

In [38]:
train_dir = os.path.dirname(s3_paths[0])
print(train_dir)

s3://sagemaker.tech-sketch.jp/allennlp_test


In [39]:
tuner.fit({"training": train_dir})

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-937428301455
INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-pytorch-190405-0722
